# Internals

## How tablite works

> To understand the limitations of any system you must first understands how it works.


When analyzing large(r) datasets most analytical operations are perform on or across few columns.

If the data hence is stored as rows, it means that every row needs to be read, the required columns filtered out, kept in memory and the results calculated.

If the data, in contrast, is stored as columns, the data can immediately be accessed as a single stream that can be read and processed.

Tablite's internal datastructure is column based, so filtering and selecting columns is a very fast.

In our test suite, the [1TB test](https://github.com/root-11/tablite/blob/master/tests/test_filereader_time.py#L102) verifies that an order file with 9 BN rows can be sliced in less than a second. We call this tolerable.

![1TB test](https://github.com/root-11/tablite/blob/master/docs/img/1TB_test.png)

If the data was in ram in column-oriented format the integer column alone would claim 297,967,747,072 bytes (297 Gb) so there are not be many PCs that can handle this in RAM.

Tablite uses the commonly available SolidStateDisk (SSD) as storage. SSD's typically deliver from 120 Mb/s and up in _linear read_. Linear read means that the data is available in a single stream. The column based format that tablite uses is hence exploiting the hardware in the best possible manner, with reading excess data that needs to be discarded, such as the row based format.

Linear reads should be compared to random reads: Amazon's CTO Werner Vogels described this as follows: Imagine you need to copy you movie collection of around 1 Terabyte from disk to another. If you do this using random reads it will take you around 20 days. If you do it using linear reads you'll be done in 90 minutes.

Tablite will therefore _appear_ "SLOW" if you're appending rows by rows. Tablite will _appear_ fast if you're doing column-by-column operations.

## Zero entropy = Zero copy

Tablite's data is organised as follows:

- Tables are dicts that point to Columns.
- Columns are wrappers for columnar operations and point to a PageHandler
- A PageHandlers manages low level data operations and have pointers to Pages
- Pages take care of datatype conversion and contain pointers to the HDF5 storage where the data is stored.

When concatenating two tables like in this operation: `table3 = table1 + table2`, the new Table that tablite creates (`table3`) will only create copies of the _references_ to the Pages that contain data. It does not copy the data. That is also why Tablite only will claim a few extra bytes in RAM and on SSD. The same applies to `table.copy()`. If there is no new information (entropy) there is no need to copy.

When you perform list comprehensionsm, such as this: `Table['a'] = [i for i in range(10,000,000)]`, Tablite will perform 10 IOP as the max page size is 1,000,000 entries.

Tablite Tables are mutable, and should you change a value, such as `Table['a'][3] = 7` the memory_manager will create a new page with the updates. Any references that point towards the "old page" will remain unchanged.

This also gives insights to *why* Tablite is so quick in the 1 TB test: Tablite only needs to do 1 linear reads the first and the last page for each column. That 22 IOPS at and your SSD will probably permit around 100,000 IOPS.

## Multiprocessing everywhere

Tablite's collection of helpers use multiprocessing whereever possible. 

The exceptions to this rule are reading and writing `ods`, `xlsx` and `xls` files.

For all other operations Tablite _in general_ performs the following operation:

1. Analyse the dataset, using the metadata in ram, to determine the optimal multi-processing task size based on number of virtual CPUs and the free memory.
2. Create a block of shared memory for task coordination. This is typically an index, such as a sort index. 
3. Launch subprocesses and queue tasks in a task queue.
4. Let the subprocess work on the tasks, by:
   1. Read the task limitations
   2. Perform the required operation (for example sort a page according the index in shared memory)
   3. Write the page to disk.
5. Finally the subprocesses run out of work and exit.
6. The main process assembles the output (a new table) using metadata only.

This approach seems to work well when there's a lot of data. When the datasets have fewer than 1,000,000 fields (columns * rows), it doesn't pay off to start the subprocesses and tablite simply performs the operation in the main process.


## Best practices?

Manage your IOPs to get the most out of Tablite

- Avoid usage row-by-row operation as each row requires 1 IO operation (IOP) and your SSD can probably max take 100,000 IO operations / second (IOPS).
- Do use `Column.extend( lots of values )` as this appends "lots of values" in a 1 IO. 
- Do use list comprehensions whenever you require row by row operation, such as `Table['a'] = [i for i in range(10,000,000)]` . In this operation there are 10 IOP as the max page size is 1,000,000 entries.
- Tables are mutable but there's really no need. Creating a new table only duplicates the coordinating metadata, so you may create new tables without having to worry about the volume of data. This: `table3 = table1 + table2` is only a few bytes in RAM and zero extra bytes on disk.


## Will I run out of memory?

You wont. Every tablite table is backed by HDF5 on disk. The memory footprint of a table is only the metadata required to know the relationships between variable names and the datastructures.

Let's do a comparison:

Let's monitor the memory and record the observations into a table!

First we get the imports, Tablite Table and psutil for measuring memory, and we create a nice table for our measurements:

In [1]:
%cd ..
from tablite import Table
import psutil, os, gc
import array
from time import process_time, sleep
process = psutil.Process(os.getpid())
baseline_memory = process.memory_info().rss

digits = 1_000_000

records = Table()
records.add_column('method')
records.add_column('memory')
records.add_column('time')
records

d:\github\tablite


#,method,memory,time
row,mixed,mixed,mixed


Let's now use the common and convenient "row" based format and create 1 million lists with 10 integers:

In [2]:

start = process_time()
L = []
for _ in range(digits):
    L.append(tuple([11 for _ in range(10)]))
end = process_time()

Now we check taskmanagers memory usage and add it to the table

In [3]:
records.add_rows(('1e6 lists w. 10 integers', process.memory_info().rss - baseline_memory, round(end-start,4)))
records


#,method,memory,time
row,str,int,float
0,1e6 lists w. 10 integers,139952128,0.5781


At this point we're using ~140 Mb to store 1 million lists with 10 items.

We now clear the list and let pythons garbage collector clean up


In [4]:
L.clear()
gc.collect()
sleep(0.1)


Let's now use a columnar format instead: We create 10 lists with 1 million integers:


In [5]:
start = process_time()
L = [[11 for i in range(digits)] for _ in range(10)]
end = process_time()


... and then we check psutil for memory usage

In [6]:
records.add_rows(('10 lists with 1e6 integers', process.memory_info().rss - baseline_memory, round(end-start,4)))
records


#,method,memory,time
row,str,int,float
0,1e6 lists w. 10 integers,139952128,0.5781
1,10 lists with 1e6 integers,86790144,0.3125


At this point we're using ~86 Mb to store the 10 lists with 1 million items in RAM.

We now clear the list and let pythons garbage collector clean up again.


In [7]:
L.clear()
gc.collect()
sleep(1)

We've thereby saved 50 Mb by avoiding the overhead from managing 1 million lists.

Q: But why didn't I just use an array? It would have even lower memory footprint.
A: First, array's don't handle None's and we get that frequently in dirty csv data.

Second, Table needs even less memory.

Okay, let's start with an array just for comparison.


In [8]:
start = process_time()
L = [array.array('i', [11 for _ in range(digits)]) for _ in range(10)]
end = process_time()



Again, we check psutils memory usage.


In [9]:
records.add_rows(('10 lists with 1e6 integers in arrays', process.memory_info().rss - baseline_memory, round(end-start,4)))
records

#,method,memory,time
row,str,int,float
0,1e6 lists w. 10 integers,139952128,0.5781
1,10 lists with 1e6 integers,86790144,0.3125
2,10 lists with 1e6 integers in arrays,46510080,0.4375


At this point we're using 60.0 Mb to store 10 lists with 1 million integers in an array.

Note that an array has to be a single data type that does not like anything else. This measurement is thereby a highly optimized case for single datatypes only. 

In [10]:

L.clear()
gc.collect()
sleep(1)



Now let's use Table:

We will first create 10 columns with 1 million integers, followed by making a copy of that table and look at both results

In [11]:

start = process_time()
t = Table()
for i in range(10):
    t.add_column(str(i), data=[11 for _ in range(digits)])
end = process_time()

gc.collect()

records.add_rows(('Table with 10 columns with 1e6 integers', process.memory_info().rss - baseline_memory, round(end-start,4)))
records



#,method,memory,time
row,str,int,float
0,1e6 lists w. 10 integers,139952128,0.5781
1,10 lists with 1e6 integers,86790144,0.3125
2,10 lists with 1e6 integers in arrays,46510080,0.4375
3,Table with 10 columns with 1e6 integers,6545408,1.5312


Next we add a copy of the tablite Table, and measure again:

In [12]:
start = process_time()
t2 = t.copy()
end = process_time()

records.add_rows(('2 Tables with 10 columns with 1e6 integers each', process.memory_info().rss - baseline_memory, round(end-start,4)))
records

#,method,memory,time
row,str,int,float
0,1e6 lists w. 10 integers,139952128,0.5781
1,10 lists with 1e6 integers,86790144,0.3125
2,10 lists with 1e6 integers in arrays,46510080,0.4375
3,Table with 10 columns with 1e6 integers,6545408,1.5312
4,2 Tables with 10 columns with 1e6 integers each,6557696,0.0469


Psutils now shows that we're using ~6.6 Mb to store 10 columns with 1 million integers for the first table, and only a few kilobytes more for the second table as only the metadata remains in pythons memory. Notice also how quick the copy is: 62.5 milliseconds.

Let's just make that memory column a bit easier to read:

In [13]:
records['RAM (Mb)'] = [ f"{round(i/1e6,1)} Mb" for i in records['memory'] ]
records

#,method,memory,time,RAM (Mb)
row,str,int,float,str
0,1e6 lists w. 10 integers,139952128,0.5781,140.0 Mb
1,10 lists with 1e6 integers,86790144,0.3125,86.8 Mb
2,10 lists with 1e6 integers in arrays,46510080,0.4375,46.5 Mb
3,Table with 10 columns with 1e6 integers,6545408,1.5312,6.5 Mb
4,2 Tables with 10 columns with 1e6 integers each,6557696,0.0469,6.6 Mb


The conclusion is comforting:

- A drop from ~140 Mb to ~7 Mb working memory.
- At this rate a projection of Tablite's Table class is roughly 18Mb per 1 Gb of data.

